<h2 align="center">Libraries</h2>

In [4]:
import os
import csv
import requests
import pandas as pd
import subprocess
from imageio_ffmpeg import get_ffmpeg_exe
import nltk
import numpy as np
import matplotlib.pyplot as plt
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

<h2 align="center">Downloads Movie Trailers</h2>
This downloads a movie's trialer from Youtube using the MovieLens dataset of trailer URLS.<br>
yt-dlp is used for downloads, stores movie_name, file_name in the trailerData.csv file

In [38]:
trailer_amount = 1000 #maybe take this amount as input

directoryy = "trailers"
os.makedirs(directoryy, exist_ok=True)
data_csv = "trailerData.csv"
if os.path.exists(data_csv):
    existing_df = pd.read_csv(data_csv)
else:
    existing_df = pd.DataFrame(columns=["file_name", "movie_name", "genre", "transcript"])



df = pd.read_csv("trailer_address.csv")

df = df.head(trailer_amount)
rows_to_append = []
for idx, row in df.iterrows():
    video_id = row['youtubeId']
    title = row['title'].replace(" ", "_").replace("/", "_")
    file_name = f"{title}_{video_id}.mp4"
    output_path = os.path.join(directoryy, file_name)
# no duplicate entries
    if not ((existing_df['file_name'] == file_name).any()):
        if not os.path.exists(output_path):
            print(f"Downloading {title}...")
            command = f'yt-dlp -f mp4 -o "{output_path}" https://www.youtube.com/watch?v={video_id}'
            subprocess.run(command, shell=True)
            if os.path.exists(output_path):#    if download succeeded
                rows_to_append.append([file_name, row['title'], "", ""])
        else:
            print(f"{title} already exists.")
            rows_to_append.append([file_name, row['title'], "", ""])

if rows_to_append:# new rows and saving
    new_df = pd.DataFrame(rows_to_append, columns=["file_name", "movie_name", "genre", "transcript"])
    updated_df = pd.concat([existing_df, new_df], ignore_index=True)
    updated_df.to_csv(data_csv, index=False)
    print(f"Updated {data_csv} with {len(rows_to_append)} entries.")
else:
    print("No new entries added to trailerData.csv")

print(f"First {trailer_amount} trailers processed.")

Updated trailerData.csv with 471 entries.
First 1000 trailers processed.


<h2 align="center">Scrape Genre's from TMDB</h2>
This scrapes the genre of a movie against it's name from The Movie Data Base(TMDB), then updates the trailerData.csv file

In [39]:
API_KEY_for_genre = '5d603ec4bd433ee159af77e58efdc97d'
TRAILERs_dir = 'trailers'
df_trailersCSV = 'trailerData.csv'
trls_URLS = 'trailer_address.csv'

def get_movie_genres(movie_title):
    """Fetch genres of a movie from TMDb."""
    search_url = 'https://api.themoviedb.org/3/search/movie'
    genre_url = 'https://api.themoviedb.org/3/genre/movie/list'
    genres_response = requests.get(genre_url, params={'api_key': API_KEY_for_genre})
    if genres_response.status_code != 200:
        return ''
    genre_dict = {g['id']: g['name'] for g in genres_response.json().get('genres', [])}
    response = requests.get(search_url, params={'api_key': API_KEY_for_genre, 'query': movie_title})
    if response.status_code != 200 or not response.json().get('results'):
        return ''
    genre_ids = response.json()['results'][0].get('genre_ids', [])
    return ','.join([genre_dict.get(gid, '') for gid in genre_ids if gid in genre_dict])

def update_empty_genres():
    df = pd.read_csv(df_trailersCSV)
    df['genre'] = df['genre'].astype(str)
    for index, row in df.iterrows():
        if pd.isna(row['genre']) or row['genre'].strip() == '':
            movie_name_raw = row['movie_name']
            movie_name_clean = movie_name_raw.split('(')[0].strip()
            genre = get_movie_genres(movie_name_clean)
            print(f"Updated genre for {movie_name_raw}: {genre}")
            df.at[index, 'genre'] = genre

    df.to_csv(df_trailersCSV, index=False)
    print(" Updated trailerData.csv with missing genres.")

if __name__ == '__main__':
    update_empty_genres()

 Updated trailerData.csv with missing genres.


### Convert the video(.mp4) files into audio(.wav) only<br>
Involves Pre-Processing on the audio files before conversion, to normalise all of them

In [40]:
AUDIO_dir = "audios"
os.makedirs(AUDIO_dir, exist_ok=True)

ffmpeg_path = get_ffmpeg_exe()

def convert_to_wav(input_path, output_path):
    cmd = [
        ffmpeg_path,
        "-i", input_path,
        "-ac", "1",        # mono
        "-ar", "16000",    # 16 kHz
        "-loglevel", "error",
        output_path
    ]
    subprocess.run(cmd, check=True)

# Load or initialize DataFrame
if os.path.exists(df_trailersCSV):
    df = pd.read_csv(df_trailersCSV)
else:
    df = pd.DataFrame(columns=['file_name', 'movie_name', 'genre', 'transcript', 'audio_file'])

# Ensure 'audio_file' column exists
if 'audio_file' not in df.columns:
    df['audio_file'] = ''

# Process trailers
for file in os.listdir(TRAILERs_dir):
    if file.endswith(".mp4"):
        input_path = os.path.join(TRAILERs_dir, file)
        audio_filename = os.path.splitext(file)[0] + ".wav"
        audio_path = os.path.join(AUDIO_dir, audio_filename)

        # Convert only if audio file doesn't exist
        if not os.path.exists(audio_path):
            try:
                print(f"Converting {file}...")
                convert_to_wav(input_path, audio_path)
                print(f"Saved as {audio_filename}")
            except subprocess.CalledProcessError as e:
                print(f"Conversion failed for {file}: {e}")
                continue
        else:
            print(f"Already converted: {audio_filename}")

        # Update CSV only if row exists and audio_file_name is missing
        row_index = df[df['file_name'] == file].index
        if not row_index.empty:
            index = row_index[0]
            if pd.isna(df.at[index, 'audio_file']) or df.at[index, 'audio_file'] == '':
                df.at[index, 'audio_file'] = audio_filename
                print(f"Updated CSV row for: {file}")
        else:
            # Add a new row if trailer exists but not listed
            df.loc[len(df)] = [file, '', '', '', audio_filename]
            print(f"Added new row for: {file}")

# Save updated CSV
df.to_csv(df_trailersCSV, index=False)
print(f"trailerData.csv updated.")

Converting 'Til_There_Was_You_(1997)_8VFQ9qzRbgI.mp4...
Saved as 'Til_There_Was_You_(1997)_8VFQ9qzRbgI.wav
Updated CSV row for: 'Til_There_Was_You_(1997)_8VFQ9qzRbgI.mp4
Converting 20,000_Leagues_Under_the_Sea_(1954)_00bin2vxgGE.mp4...
Saved as 20,000_Leagues_Under_the_Sea_(1954)_00bin2vxgGE.wav
Updated CSV row for: 20,000_Leagues_Under_the_Sea_(1954)_00bin2vxgGE.mp4
Converting 2001#_A_Space_Odyssey_(1968)_bAZgLxuBtbo.mp4...
Saved as 2001#_A_Space_Odyssey_(1968)_bAZgLxuBtbo.wav
Added new row for: 2001#_A_Space_Odyssey_(1968)_bAZgLxuBtbo.mp4
Converting 2_Days_in_the_Valley_(1996)_umjwm08s104.mp4...
Saved as 2_Days_in_the_Valley_(1996)_umjwm08s104.wav
Updated CSV row for: 2_Days_in_the_Valley_(1996)_umjwm08s104.mp4
Converting 39_Steps,_The_(1935)_hUIL0Z1c9bc.mp4...
Saved as 39_Steps,_The_(1935)_hUIL0Z1c9bc.wav
Updated CSV row for: 39_Steps,_The_(1935)_hUIL0Z1c9bc.mp4
Converting Above_the_Rim_(1994)_sEsCXWD7-Cc.mp4...
Saved as Above_the_Rim_(1994)_sEsCXWD7-Cc.wav
Updated CSV row for: Abov

## Using ASR model for generating transcripts

In [2]:
import os
import wave
import json
import pandas as pd
from vosk import Model, KaldiRecognizer

def transcribe_audio(audio_file_path):
    model_path = "vosk-model-small-en-us-0.15"

    wf = wave.open(audio_file_path, "rb")
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() != 16000:
        raise ValueError(f"{audio_file_path} must be mono, 16-bit, 16kHz")

    model = Model(model_path)
    rec = KaldiRecognizer(model, wf.getframerate())

    results = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            results.append(json.loads(rec.Result()))
    results.append(json.loads(rec.FinalResult()))

    transcript = " ".join(r.get("text", "") for r in results)
    return transcript

def process_all_audio_files(csv_path="trailerData.csv", audio_folder="audios"):
    a=0
    df = pd.read_csv(csv_path)

    if 'transcript' not in df.columns:
        df['transcript'] = ""

    model_path = "vosk-model-small-en-us-0.15"
    model = Model(model_path)

    for idx, row in df.iterrows():
        audio_filename = row['audio_file']
        audio_path = os.path.join(audio_folder, audio_filename)

        if not os.path.isfile(audio_path):
            print(f"[SKIPPED] File not found: {audio_filename}")
            continue

        # print(f"[PROCESSING] {audio_filename}")
        try:
            wf = wave.open(audio_path, "rb")
            if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() != 16000:
                print(f"[SKIPPED] Invalid format: {audio_filename}")
                continue

            rec = KaldiRecognizer(model, wf.getframerate())
            results = []

            while True:
                data = wf.readframes(4000)
                if len(data) == 0:
                    break
                if rec.AcceptWaveform(data):
                    results.append(json.loads(rec.Result()))
            results.append(json.loads(rec.FinalResult()))

            transcript = " ".join(r.get("text", "") for r in results)
            df.at[idx, 'transcript'] = transcript
            print(f"[DONE] Transcript added for: {audio_filename},       {a} done")
            a+=1

        except Exception as e:
            print(f"[ERROR] {audio_filename}: {e}")

    df.to_csv(csv_path, index=False)
    print("[FINISHED] CSV updated.")

# Run the full processing
process_all_audio_files()


C:\Users\Ibrahim\AppData\Local\Temp\ipykernel_13784\1720969494.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'sergeant establish your recount post downstairs cobra repeat we are can grab recount plant charlie execute know whoop it up walt disney pictures presents a totally new animated motion picture event star command come in the read me the story of to toy there seems to be no sign of intelligent anywhere else oh yeah pay for a showdown my name is woody this is my spot now i am buzz lightyear i come in peace you are a child's play you are a sad strange little man and playing by their own rules wrong me again i don't like confrontations yeah buyer your money in the original oh impressive wingspan very good why why he can't fly yes i can't can't can't can't can't this holiday season the adventure takes on it when toys come to mind no and video and meal toy story yeah ' has dtype incompatible with f

[DONE] Transcript added for: Toy_Story_(1995)_K26_sDKnvMU.wav,       0 done
[DONE] Transcript added for: Grumpier_Old_Men_(1995)_rEnOoWs3FuA.wav,       1 done
[DONE] Transcript added for: Waiting_to_Exhale_(1995)_j9xml1CxgXI.wav,       2 done
[DONE] Transcript added for: Heat_(1995)_2GfZl4kuVNI.wav,       3 done
[DONE] Transcript added for: Sabrina_(1995)_twTksx_lWB4.wav,       4 done
[DONE] Transcript added for: Tom_and_Huck_(1995)_-C-xXZyX2zU.wav,       5 done
[DONE] Transcript added for: GoldenEye_(1995)_lcOqUE0u1LM.wav,       6 done
[DONE] Transcript added for: Nixon_(1995)_dO2LWKpeyI8.wav,       7 done
[DONE] Transcript added for: Casino_(1995)_EJXDMwGWhoA.wav,       8 done
[DONE] Transcript added for: Get_Shorty_(1995)_yNLaTtpovys.wav,       9 done
[DONE] Transcript added for: Copycat_(1995)_lsmXhM4yfU0.wav,       10 done
[DONE] Transcript added for: Assassins_(1995)_OOTTvrb4JZI.wav,       11 done
[DONE] Transcript added for: Powder_(1995)_gHl-UHu2-lM.wav,       12 done
[DONE] Tr

## Transcript Cleaning

In [ ]:
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

transcript = "cgvbhnjkm, fvygbhnjm gybhunjm gyubhnji tgybuhnjm uhbnjimk fvbn"
transcript = transcript.lower()
punctuation_e = string.punctuation
punctuation_ = punctuation_e+"1234567890"
for a in transcript:
    if a in punctuation_:
        transcript = transcript.replace(a, " ")

tokenssss = nltk.word_tokenize(transcript)
stop_words = set(stopwords.words('english'))
alternate = tokenssss
for b in tokenssss:
    if b in stop_words:
        alternate.remove(b)

lemmat = alternate

lemmatizer = WordNetLemmatizer()
welp = lemmat
for c in lemmat:
    welp[lemmat.index(c)] = lemmatizer.lemmatize(c)


# take every transcript from trailerData.csv with the cleaned transcript that is made here and 
# put it in trainingData.csv(its an empty file but needs to be appended) along with movie name and its genre 
# then take all action movie transcripts and append them in action.txt(also check if file exists or not, if it doesnot exist 
# then make it and put the words 'action' first, and similarly for every genre
# and then append it in there, basically i want to group and have a whole sets of transcripts of every genre in its own txt file),
# so like this for every genre and it should be for every genre found in the csv file

# use unsupervised machine learning for classifiying the transcripts into genres, basically its like
# making clusters of transcripts like making k-clusters of transcripts and every cluster represents a genre 
# and the transcripts must go through sentiment analysis so that on basis of the sentiment should the classification be done 

SyntaxError: invalid syntax (1018865281.py, line 31)